<a href="https://colab.research.google.com/github/davidfague/Stylized-ReducedOrder-L5-Model/blob/main/Passive_Properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Neuron Initialization
Import modules and setup simulation parameters

In [1]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Stylized-Cell-model/

/content/drive/MyDrive/Stylized-Cell-model


In [4]:
from neuron import h
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import os
from typing import Optional, List, Tuple, Union

In [5]:


from stylized_module.stylized_cell import Stylized_Cell
from stylized_module.ecp import EcpMod, newposition
from stylized_module.recorder import Recorder
from stylized_module.point_current import Point_current

mechanisms_path = 'mechanisms'  # need to compile the folder first
geometry_file = 'geom_parameters.csv'
tstop = 1000  # ms simulation duration
dt = 0.025  # ms. does not allow change

h.load_file('stdrun.hoc')
#h.nrn_load_dll(os.path.join(mechanisms_path,'nrnmech.dll'))  # running this line more than once might crash the kernel
h.nrn_load_dll(os.path.join(mechanisms_path,'x86_64/.libs/libnrnmech.so'))  # use this line instead in Linux
geo_standard = pd.read_csv(geometry_file,index_col='id')
h.tstop = tstop
h.dt = dt

In [6]:
#pick a random seed to initalize random values
import random
random.seed(427)
pi=np.pi

In [7]:
#generate lognormal distribution
m=.2
s=0.345
mean = np.log(m) - 0.5 * np.log((s/m)**2+1)
std = np.sqrt(np.log((s/m)**2 + 1))
print(mean)
print(std)

-2.299528896659566
1.1748114608101725


Define cell model and simulation model. Edit method "set_channels" to design biophysical properties.

In [8]:
class Cell(Stylized_Cell):
    """Define single cell model using parent class Stylized_Cell"""
    def __init__(self,geometry=None,biophys=None,dL=30,vrest=-60.0): #-90?
        """
        Initialize cell model
        geometry: pandas dataframe of cell morphology properties
        biophys: vector of biophysical parameters corresponding to "biophys_entries". Use -1 for default value.
        dL: maximum segment length
        vrest: reversal potential for leak channels
        """
        self.biophys = biophys
        super().__init__(geometry,dL,vrest)
        self.record_soma_v() # uncomment this if want to record soma voltage
    
    def set_channels(self):
        """Define biophysical properties, insert channels"""
        self.define_biophys_entries()
        # common parameters
        for sec in self.all:
            sec.cm = 2.0
            sec.Ra = 100
            sec.insert('pas')
            sec.e_pas = self._vrest
        # fixed parameters
        soma = self.soma
        soma.cm = 1.0

        

        for isec in self.grp_ids[1]:        # Basal, SomaApical, ProxTrunk and ProxApical dendrites
            sec = self.get_sec_by_id(isec) 


        for isec in self.grp_ids[2]:
            sec = self.get_sec_by_id(isec)  # Mid Trunk


        for isec in self.grp_ids[3]:
            sec = self.get_sec_by_id(isec)  # Distal Trunk

        for isec in self.grp_ids[4]:
            sec = self.get_sec_by_id(isec)  # Tuft dendrites




        # variable parameters
        for i,entry in enumerate(self.biophys_entries):
            for sec in self.get_sec_by_id(self.grp_ids[entry[0]]):
                setattr(sec,entry[1],self.biophys[i])
        h.v_init = self._vrest
    

    def define_biophys_entries(self):
        """
        Define list of entries of biophysical parameters.
        Each entry is a pair of group id and parameter reference string.
        Define default values and set parameters in "biophys".
        """
        self.grp_sec_type_ids = [[0],[1,2,3],[4],[5],[6]]  # select section id's for each group
        self.grp_ids = []  # get indices of sections for each group
        for ids in self.grp_sec_type_ids:
            secs = []
            for i in ids:
                secs.extend(self.sec_id_lookup[i])
            self.grp_ids.append(secs)
        self.biophys_entries = [
             (0,'g_pas'),(1,'g_pas'),(2,'g_pas'),(3,'g_pas'),(4,'g_pas')  # g_pas of soma, basal, mid trunk, distal trunk, and tuft 

        ]

        default_biophys = np.array([0.0000338,0.0000467,0.0000589,0.0000589,0.0000589 ## g_pas of soma, basal, midTrunk, distal trunk, and tuft 

                                    ])
        



        if self.biophys is not None:
            for i in range(len(self.biophys)):
                if self.biophys[i]>=0:
                    default_biophys[i]=self.biophys[i]
        self.biophys = default_biophys
    

    def add_synapse(self,stim,sec_index,**kwargs):
        """Add synapse to a section by its index"""
        self.injection.append(Synapse(self,stim,sec_index,**kwargs))
 

    def record_soma_v(self):
        self.v_rec = Recorder(self.soma(.5),'v')
    
    def v(self):
        """Return recorded soma membrane voltage in numpy array"""
        if hasattr(self,'v_rec'):

            return self.v_rec.as_numpy()

class Synapse(Point_current):
    def __init__(self,cell,stim,sec_index,gmaxEXC=1,gmaxINH=1,loc=0.5,record=True, SynType='exc'): 
        super().__init__(cell,sec_index,loc)
        self.stim = stim
        self.gmaxEXC = gmaxEXC
        self.gmaxINH = gmaxINH
        self.syntype = SynType
        self.setup(record,loc)

        

    def setup(self,record,loc):
        self.setup_synapse(loc)
        if record:
            self.setup_recorder()
    
    def setup_synapse(self,loc): 
        if self.syntype == 'inh':
          self.pp_obj = h.int2pyr(self.get_section()(loc))
          #Assigns random generator of release probability.
          r = h.Random()
          r.MCellRan4()
          r.uniform(0,1)
          self.pp_obj.setRandObjRef(r)
          #A list of random generators is kept so that they are not automatically garbaged.
          self.generator = r
          syn = self.pp_obj
          syn.setRandObjRef(r)
          syn.P_0 = 0.6
          syn.initW = self.gmaxEXC
          self.nc = h.NetCon(self.stim,syn,1,0,1)

        else:
          self.pp_obj = h.pyr2pyr(self.get_section()(loc))
          #Assigns random generator of release probability.
          r = h.Random()
          r.MCellRan4()
          r.uniform(0,1)
          self.pp_obj.setRandObjRef(r)
          #A list of random generators is kept so that they are not automatically garbaged.
          self.generator = r
          syn = self.pp_obj
          syn.setRandObjRef(r)
          syn.P_0 = 0.6
          syn.initW = self.gmaxINH
          self.nc = h.NetCon(self.stim,syn,1,0,1)

    def setup_recorder(self):
          size = [round(h.tstop/h.dt)+1] if hasattr(h,'tstop') else []
          try:
              self.rec_vec = h.Vector(*size).record(self.pp_obj._ref_igaba)
          except:
                  self.rec_vec = MultiSynCurrent()
                  vec_inmda = h.Vector(*size).record(self.pp_obj._ref_inmda)
                  vec_iampa = h.Vector(*size).record(self.pp_obj._ref_iampa)
                  self.rec_vec.add_vec(vec_inmda)
                  self.rec_vec.add_vec(vec_iampa)
                  

class MultiSynCurrent(object):
    def __init__(self):
        self.vec_list = []
    
    def add_vec(self,vec):
        self.vec_list.append(vec)
    
    def as_numpy(self):
        return np.sum(np.array([vec.as_numpy() for vec in self.vec_list]), axis=0)


class Simulation(object):
    def __init__(self,geometry,electrodes,loc_param=[0.,0.,0.,0.,1.,0.],
                 geo_param=[-1],biophys=[-1],
                 gmaxEXC=0.015,gmaxINH=0.015,
                 scale=1.0,ncell=1,exc_dens=2.16,
                 spike_threshold: Optional[float] = None,
                 inh_dens=0.22): #initially __init__(self,geometry,electrodes,loc_param=[0.,0.,0.,0.,1.,0.],geo_param=[-1],biophys=[-1],gmax=0.015,scale=1.0,ncell=1):
        """
        Initialize simulation object
        geometry: pandas dataframe of cell morphology properties
        electrodes: array of electrode coordinates, n-by-3
        loc_param: location parameters, ncell-by-6 array, (x,y,z,theta,h,phi)
        geo_param: geometry parameters, ncell-by-k array, if not specified, use default properties in geometry
        biophys: biophysical parameters, ncell-by-k array, if not specified, use default properties
        gmax: maximum conductance of synapse, ncell-vector, if is single value, is constant for all cells
        scale: scaling factors of lfp magnitude, ncell-vector, if is single value, is constant for all cells
        ncell: number of cells in the simulation, required if simulating for multiple cells
        """
        self.ncell = ncell  # number of cells in this simulation
        self.cells = []  # list of cell object
        self.lfp = []  # list of EcpMod object
        self.define_geometry_entries()  # list of entries to geometry dataframe
        self.geometry = geometry.copy()
        self.electrodes = electrodes
        self.set_loc_param(loc_param)
        self.set_geo_param(geo_param)
        self.set_biophys(biophys)
        self.set_gmax(gmaxEXC, gmaxINH)
        self.set_scale(scale)
        self.create_cells(exc_dens,inh_dens)  # create cell objects with properties set up
        self.t_vec = h.Vector( round(h.tstop/h.dt)+1 ).record(h._ref_t)  # record time
        self.spike_threshold = None
        self.record_spikes(spike_threshold)


    def set_netstim(self, syn_number):
        """Setup synaptic input event"""
        stim_list = []
        self.stim = stim_list
        for i in range(syn_number):
          stim = h.NetStim()
          stim.number = 2  # average number of spikes
          stim.start = 50  #  most likely start time of first spike
          stim.noise = 1  # fractional randomness
          stim.interval = 500  # mean time between spikes (ms)
          #duration of stimulation = stim.interval X stim.number
          stim_list.append(stim)


    
    def pack_parameters(self,param,ndim,param_name):
        """Pack parameters for the simulation"""
        if ndim==0:
            if not hasattr(param,'__len__'):
                param = [param]
            param = np.array(param).ravel()
            if param.size!=self.ncell:
                if param.size==1:
                    param = np.broadcast_to(param,self.ncell)
                else:
                    raise ValueError(param_name+" size does not match ncell")   
        if ndim==1:
            param = np.array(param)
            if param.ndim==1:
                param = np.expand_dims(param,0)
            if param.shape[0]!=self.ncell:
                if param.shape[0]==1:
                    param = np.broadcast_to(param,(self.ncell,param.shape[1]))
                else:
                    raise ValueError(param_name+" number of rows does not match ncell")
        return param
    
    def set_loc_param(self,loc_param):
        """Setup location parameters. loc_param ncell-by-6 array"""
        loc_param = self.pack_parameters(loc_param,1,"loc_param")
        self.loc_param = [(loc_param[i,:3],loc_param[i,3:]) for i in range(self.ncell)]
    
    def set_geo_param(self,geo_param):
        """Setup geometry parameters. geo_param ncell-by-k array, k entries of properties"""
        self.geo_param = self.pack_parameters(geo_param,1,"geo_param")
    
    def set_biophys(self,biophys):
        """Setup geometry parameters. geo_param ncell-by-k array, k entries of properties"""
        self.biophys = self.pack_parameters(biophys,1,"biophys")
    
    def set_gmax(self,gmaxEXC,gmaxINH):
        """Setup maximum conductance of synapse"""
        self.gmaxEXC = self.pack_parameters(gmaxEXC,0,"gmaxEXC")
        self.gmaxINH = self.pack_parameters(gmaxINH,0,"gmaxINH")
    
    def set_scale(self,scale):
        """setup scaling factors of lfp magnitude"""
        self.scale = self.pack_parameters(scale,0,"scale")
    
    def define_geometry_entries(self):
        """Define list of entries to geometry dataframe. Each entry is a pair of section id and property."""
        self.geo_entries = [
            (0,'R'),  # change soma radius
            (3,'L'),  # change trunk length
            (3,'R'),  # change trunk radius
            ([1,2],'R'),  # change basal dendrites radius
            (4,'R'),  # change tuft radius
            ([1,2],'L'), # change basal,apical dendrites length
            (4,'L'), #change tuft dendrites length
            (4,'ang') # change tuft angle
        ]
    
    def set_geometry(self,geometry,geo_param):
        """Set property values from geo_param through each entry to geometry. Return dataframe"""
        geom = geometry.copy()
        for i,x in enumerate(geo_param):
            if x>=0:
                geom.loc[self.geo_entries[i]] = x
        return geom

    def record_spikes(self, threshold: Optional[float]) -> None:
        """Setup spike recorder for all cells"""
        for cell in self.cells:
            cell.set_spike_recorder(threshold)
        self.spike_threshold = threshold

    def create_cells(self,exc_dens,inh_dens):
        """Create cell objects with properties set up"""
        self.cells.clear()  # remove cell objects from previous run
        self.lfp.clear()
        # Create cell with morphology and biophysical parameters
        for i in range(self.ncell):
            geometry = self.set_geometry(self.geometry,self.geo_param[i,:])
            self.cells.append( Cell(geometry=geometry,biophys=self.biophys[i,:]) )
        # add injection current or synaptic current and set up lfp recording
        min_dist = 10.0 # minimum distance allowed between segment and electrode. Set to None if not using.
        
        





        for i,cell in enumerate(self.cells):
          self.lfp.append( EcpMod(cell,self.electrodes,move_cell=self.loc_param[i],scale=self.scale[i],min_distance=min_dist) )
          # Current injection in soma
          cell.add_injection(sec_index=0,record=True,delay=150,dur=850,amp=-0.1) # Tune for proper action potential 




          for id,sec in self.geometry.iterrows():
            sec_index_list=cell.sec_id_lookup[id]
            L = sec['L']
            #print(sec_index_list)

            if id==0: # soma
              print('soma skipped') #skip soma
              ### add equivalent of 150 inh synapses to soma
              #syn_number = 6
              #self.set_netstim(syn_number)
              #print(self.stim)
              #syn_dist = np.random.rand(syn_number) #generate distribution for location
              #gmax_dist = np.random.lognormal(mean, std, syn_number) #generate distribution for gmax
              #for j in range(syn_number):
              #cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[0],gmaxEXC=gmax_dist[j],gmaxINH=1,loc=syn_dist[j],SynType='inh')

            elif id==1 or id == 2 or id == 3: # basal, somaapical, proxtrunk
              print('basal, somapical, proxtrunk skipped') #skip basal



            elif id==4: # proxapical
              print('proxapical skipped') #skip proximal apical dendrites

            elif id==5: # midtrunk       
              print('midtrunk skipped') #skip midsection of trunk

            elif id==6: # disttrunk     
              print('disttrunk skipped') #skip distal section of trunk

            elif id==7: # tufts
              print('tufts skipped') #skip tuft
              
            else: # add synapses to each section
              
              syn_exc_number = np.round(exc_dens*L) # number of synapses on dendrites = density (syn/um) times length of dendrites (um)
              syn_inh_number = np.round(inh_dens*L)
              syn_number = int(syn_inh_number + syn_exc_number) #total number of synapses
              print(syn_number)
              prop_exc = syn_exc_number/syn_number #used for assigning type
              for isec in range(len(sec_index_list)):
                syn_dist = np.random.rand(syn_number) #generate distribution for location
                rand_syntype = np.random.rand(syn_number)  #generate distribution for type
                gmax_dist = np.random.lognormal(mean, std, syn_number) #generate distribution for gmax
                syn_type_list = ['exc' if y< prop_exc else #assign randomly, adhere to proportion 
                                'inh' for y in rand_syntype]
                self.set_netstim(syn_number) #generate spike train list for synapses
                #print(self.stim)
                for j in range(syn_number):
                  cell.add_synapse(stim=self.stim[j],sec_index=sec_index_list[isec],gmaxEXC=gmax_dist[j],gmaxINH=1,loc=syn_dist[j],SynType=syn_type_list[j])
        

        




    
    def run(self):
        """Run simulation"""
        h.run()
    
    def t(self):
        """Return simulation time vector"""
        return self.t_vec.as_numpy()
    
    def get_lfp(self,index=0):
        """Return LFP array of the cell by index (indices), (cells-by-)channels-by-time"""
        if not hasattr(index,'__len__'):
            lfp = self.lfp[index].calc_ecp()
        else:
            index = np.asarray(index).ravel()
            lfp = np.stack([self.lfp[i].calc_ecp() for i in index],axis=0)
        return lfp

    def get_spike_time(self, index: Union[np.ndarray, List[int], int, str] = 0) -> np.ndarray:
        """
        Return soma spike time of the cell by index (indices), ndarray (list of ndarray)
        Parameters
        index: index of the cell to retrieve the spikes from
        """
        if self.spike_threshold is None:
            raise ValueError("Spike recorder was not set up.")
        if type(index) is str and index == 'all':
            index = range(self.ncell)
        if not hasattr(index, '__len__'):
            spk = self.cells[index].spikes.as_numpy().copy()
        else:
            index = np.asarray(index).ravel()
            spk = np.array([self.cells[i].spikes.as_numpy().copy() for i in index], dtype=object)
        return spk

    def get_spike_number(self, index: Union[np.ndarray, List[int], int, str] = 0) -> Union[int, np.ndarray]:
        """
        Return soma spike number of the cell by index (indices), int (ndarray)
        Parameters
        index: index of the cell to retrieve the spikes from
        """
        if self.spike_threshold is None:
            raise ValueError("Spike recorder was not set up.")
        if index == 'all':
            index = range(self.ncell)
        if not hasattr(index, '__len__'):
            spk = self.get_spike_time(index)
            nspk = spk.size
        else:
            index = np.asarray(index).ravel()
            spk = self.get_spike_time(index)
            nspk = np.array([s.size for s in spk])
        return nspk, 

#### Create simulation

In [9]:
x = np.zeros(96)
y = np.linspace(-1900,1900,96)
z = np.zeros(96)
elec_pos = pos = np.column_stack((x,y,z))

Set parameters

In [10]:
loc_param = [50,0,10,0,1,0]  # example position #D: initially this was [0,0,50,0.2,0.75,1.5], (x,y,z,theta,h,phi), h=[-1,1] phi=[-pi,pi] h=.7071067812 for 90 deg
#note on parameters: Rotate(alpha,h,phi): first rotate alpha about y-axis (spin), then rotate arccos(h) about x-axis (elevation), then rotate phi about y axis (azimuth).

#geo_param = [-1,-1,-1]  # choose trunk length [-1,600], bounds([3,12],[20,800],[0.2,1.0],[0.2,1.0],[0.2,1.0],[100,300])
geo_param = [ # geo_entries need to be fixed
            -1,  # change soma radius
            -1,  # change trunk length
            -1,  # change trunk radius
            -1,  # change basal dendrites radius
            -1,  # change tuft radius
            -1, # change basal, apical dendrites length
            -1, # change tuft dendrites length
            -1 # change tuft angle # 1.309
        ]

# example of changing biophysical parameters (check "biophys_entries" in "Cell" class for meaning, bounds can also be found there)
biophys = [0.0000338,0.0000467,0.0000589,0.0000589,0.0000589 # g_pas of soma, basal, midTrunk, distal trunk, and tuft 0.0000338,0.0000467,0.0000589,0.0000589,0.0000589,

                                    ]


                              
sim = Simulation(geo_standard,elec_pos,loc_param,geo_param=geo_param,biophys=biophys,gmaxEXC=0.1,gmaxINH=0.05,scale=100.,exc_dens=2.16,inh_dens=0.22,spike_threshold = 10)  # 0.001 -0.012 #gmax=.005, scale=100

soma skipped
basal, somapical, proxtrunk skipped
basal, somapical, proxtrunk skipped
basal, somapical, proxtrunk skipped
proxapical skipped
midtrunk skipped
disttrunk skipped
tufts skipped


Record membrane voltage at initial/middle/end of the trunk and middle of the tuft to check the propagation.

In [11]:
nseg = len(sim.cells[0].segments)
numTstep = int(tstop/dt)


seg_list = sim.cells[0].get_seg_by_id(range(nseg))

Vrecord = []
for i in range(0,len(seg_list)):
     Vrecord.append(seg_list[i])

dend_v = Recorder(Vrecord)





In [12]:
sim.run()
lfp = sim.get_lfp().T
t = sim.t()

In [15]:
v_dend = dend_v.as_numpy()

v_rest = v_dend[0][1]
v_final = v_dend[0][40000]


calc = v_rest -((v_rest - v_final) * 0.638)

print(calc)


print(v_dend[0][7289])

timeAtV = 7289/40
print(timeAtV)

t = timeAtV - 150
t_ins = t/1000

print(t)
print(t_ins)


deltaV = (v_rest - v_final)
deltaI = (0 - (-100))
dVdI = deltaV / (deltaI /1000)

print(dVdI)


-69.78461003146845
-69.78503612976816
182.225
32.224999999999994
0.032225
153.36379359668413




1.  V_rest = -60 ms
2.  Calculation of time constant:
*   Start inject: 150ms / -60mV 
*   Final Value: ~ -75.34mV
*   Difference: -15.34   | 63.2% = 9.78   | -60 – 8.63 = - 69.78
*   Time at – 69.78: 170.375 ms
*   τ = 182.225-150 
*   τ  = 32.2249 ms
*   τ  = .032225 s

3.  Input Resistance 

  ΔV/ΔI = ( -60 – (-92.24) )/( 0 – (-100pA) ) 

  = 15.34mV / 0.1 nA 

  R_in   = 153.364 MΩ
